# Re-encode inputs

Starting from data that includes a column for each relevant condition

The format of the numpy array that we want is:

[ whole data [ one user [ one day ] ] ]

If weather is not in features: one day: [exercise, stress, gluten, sugar, alcohol, dairy, antibiotic, caffeine, meat, low fod map, condition1, condition 2, ...]


_Note: fiber is not included_

In [37]:
import pandas as pd
import numpy as np

In [38]:
# connect to s3 bucket
from sagemaker import get_execution_role
import boto3
bucket = 'montrealgr2'
role = get_execution_role()

In [39]:
#Read the dataset
data_key = 'data_with_condition_columns.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
df = pd.read_csv(data_location)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
df

Unnamed: 0  Unnamed: 0.1  user_id   age     sex country  \
0                 0             0        1  30.0  female      BE   
1                 1             1        1  30.0  female      BE   
2                 2             2        1  30.0  female      BE   
3                 3             3        1  30.0  female      BE   
4                 4             4        1  30.0  female      BE   
...             ...           ...      ...   ...     ...     ...   
3553751     3553751       3553751    11478  48.0    male      US   
3553752     3553752       3553752    11478  48.0    male      US   
3553753     3553753       3553753    11478  48.0    male      US   
3553754     3553754       3553754    11478  48.0    male      US   
3553755     3553755       3553755    11478  48.0    male      US   

         checkin_date trackable_id trackable_type            trackable_name  \
0                   1         1288      Condition                 Leaky Gut   
1                   1          649      Condition                  Migraine   
2                   1         1022      Condition  Thoracic outlet syndrome   
3                   1          886      Condition      Rheumatoid Arthritis   
4                   1          397      Condition              Fibromyalgia   
...               ...          ...            ...                       ...   
3553751            12       175150        Weather           temperature_min   
3553752            12       175150        Weather           temperature_max   
3553753            12       175150        Weather          precip_intensity   
3553754            12       175150        Weather                  pressure   
3553755            12       175150        Weather                  humidity   

         ... IBS  IBD  Chron's  Lupus  Rheumatoid Arthritis  Fibromyalgia  \
0        ...   0    0        0      0                     0             0   
1        ...   0    0        0      0                     0             0   
2        ...   0    0        0      0                     0             0   
3        ...   0    0        0      0                     1             0   
4        ...   0    0        0      0                     0             1   
...      ...  ..  ...      ...    ...                   ...           ...   
3553751  ...   0    0        0      0                     0             0   
3553752  ...   0    0        0      0                     0             0   
3553753  ...   0    0        0      0                     0             0   
3553754  ...   0    0        0      0                     0             0   
3553755  ...   0    0        0      0                     0             0   

         Dysautonomia  Arthritis  Psoriasis  Raynaud's Phenomenon  
0                   0          0          0                     0  
1                   0          0          0                     0  
2                   0          0          0                     0  
3                   0          0          0                     0  
4                   0          0          0                     0  
...               ...        ...        ...                   ...  
3553751             0          0          0                     0  
3553752             0          0          0                     0  
3553753             0          0          0                     0  
3553754             0          0          0                     0  
3553755             0          0          0                     0  

[3553756 rows x 22 columns]

## Drop all the rows that have trackable_types that we don't care about

In [41]:
# create a list of our features from chosen_features.txt
FEATURES = []
features_file = open("chosen_features.txt","r")
for feature in features_file:
    clean_feature = feature.strip('\n')
    FEATURES.append(clean_feature)

In [42]:
if 'Weather' in FEATURES:
    unwanted_trackable_types = ['Symptom', 'Treatment', 'HBI']
else : 
    unwanted_trackable_types = ['Symptom', 'Treatment', 'Weather', 'HBI']

our_trackable_types_df = df.copy(deep=True)

for unwanted in unwanted_trackable_types:
    our_trackable_types_df = our_trackable_types_df[our_trackable_types_df['trackable_type'] != unwanted]

In [43]:
our_trackable_types_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'trackable_id', 'country'], axis=1)

user_id   age     sex  checkin_date trackable_type  \
0              1  30.0  female             1      Condition   
1              1  30.0  female             1      Condition   
2              1  30.0  female             1      Condition   
3              1  30.0  female             1      Condition   
4              1  30.0  female             1      Condition   
...          ...   ...     ...           ...            ...   
3553728    11478  48.0    male            12      Condition   
3553729    11478  48.0    male            12      Condition   
3553730    11478  48.0    male            12      Condition   
3553731    11478  48.0    male            12      Condition   
3553732    11478  48.0    male            12      Condition   

                          trackable_name trackable_value  Scleroderma  IBS  \
0                              Leaky Gut               0            0    0   
1                               Migraine               0            0    0   
2               Thoracic outlet syndrome               1            0    0   
3                   Rheumatoid Arthritis               1            0    0   
4                           Fibromyalgia               1            0    0   
...                                  ...             ...          ...  ...   
3553728                  chronic fatigue               1            0    0   
3553729   treatment resistant depression               1            0    0   
3553730                          Anxiety               1            0    0   
3553731  Borderline personality disorder               1            0    0   
3553732                     Fibromyalgia               1            0    0   

         IBD  Chron's  Lupus  Rheumatoid Arthritis  Fibromyalgia  \
0          0        0      0                     0             0   
1          0        0      0                     0             0   
2          0        0      0                     0             0   
3          0        0      0                     1             0   
4          0        0      0                     0             1   
...      ...      ...    ...                   ...           ...   
3553728    0        0      0                     0             0   
3553729    0        0      0                     0             0   
3553730    0        0      0                     0             0   
3553731    0        0      0                     0             0   
3553732    0        0      0                     0             1   

         Dysautonomia  Arthritis  Psoriasis  Raynaud's Phenomenon  
0                   0          0          0                     0  
1                   0          0          0                     0  
2                   0          0          0                     0  
3                   0          0          0                     0  
4                   0          0          0                     0  
...               ...        ...        ...                   ...  
3553728             0          0          0                     0  
3553729             0          0          0                     0  
3553730             0          0          0                     0  
3553731             0          0          0                     0  
3553732             0          0          0                     0  

[952360 rows x 18 columns]

## Fix the user_ids in the dataframe

In [44]:
# Currently, the biggest user_id does not fit the total number of unique user_id's:
num_user_ids = our_trackable_types_df["user_id"].nunique()
print ("num_user_ids:", num_user_ids)
max_user_id = our_trackable_types_df["user_id"].max()
print ("max_user_id:", max_user_id)

num_user_ids: 11470
max_user_id: 11478


In [45]:
# Reassign correct user_id's
correct_ids = list(range(1, num_user_ids+1)) # Once we go through all the rows of one user, we remove this user_id from this list 
error = 0 # The number of ids that were found missing
for i, row in our_trackable_types_df.iterrows():
    cur_id = our_trackable_types_df.at[i,'user_id']
    our_trackable_types_df.at[i,'user_id'] = cur_id - error
    cur_id = our_trackable_types_df.at[i,'user_id']
    
    if cur_id == correct_ids[0]: # Keep as is
        continue
    elif cur_id == correct_ids[1]: # Remove correct_ids[0] so that now cur_id==correct_ids[0]
        correct_ids.pop(0) 
    else : # if an id is skipped
        our_trackable_types_df.at[i,'user_id'] = cur_id - 1
        correct_ids.pop(0)
        error += 1

In [46]:
# Check that now the biggest user_id is equal to the total number of unique user_id's
num_user_ids = our_trackable_types_df["user_id"].nunique()
print ("num_user_ids:", num_user_ids)
max_user_id = our_trackable_types_df["user_id"].max()
print ("max_user_id:", max_user_id)

num_user_ids: 11470
max_user_id: 11470


## Create a numpy array to store the data

In [47]:
# Determine array dimensions

# (1) number of users
NUM_USERS = our_trackable_types_df['user_id'].nunique()

# (2) number of days per user
# Issue: each user does not have the same amount of days
MAX_NUM_DAYS = our_trackable_types_df['checkin_date'].max()
# Every user must have MAX_NUM_DAYS entries. We will add the missing entries and put all values to 0 (in a loop?).

# (3) number of features and labels
FEATURES_NUM = len(FEATURES)

# create a list of our labels from chosen_conditions.txt
LABELS = []
cond_file = open("chosen_conditions.txt","r")
for cond in cond_file:
    clean_cond = cond.strip('\n')
    LABELS.append(clean_cond)
LABELS_NUM = len(LABELS)

In [48]:
# create numpy array to store the whole data (initialized with zeros)
X_array = np.zeros(shape=(NUM_USERS, MAX_NUM_DAYS, FEATURES_NUM))
Y_array = np.zeros(shape=(NUM_USERS, MAX_NUM_DAYS, LABELS_NUM))

## Add info to X_array and Y_array

For verification, check if dimensions match:

In [49]:
print ("inputarray shape:", X_array.shape)
print ("output array shape:", Y_array.shape)

num_user_ids = our_trackable_types_df["user_id"].nunique()
print ("num_user_ids:", num_user_ids)
max_user_id = our_trackable_types_df["user_id"].max()
print ("max_user_id:", max_user_id)

num_dates = our_trackable_types_df["checkin_date"].nunique()
print ("num_dates:", num_dates)
max_date = our_trackable_types_df["checkin_date"].max()
print ("max_date:", max_date)

inputarray shape: (11470, 1621, 10)
output array shape: (11470, 1621, 11)
num_user_ids: 11470
max_user_id: 11470
num_dates: 1621
max_date: 1621


### Create dictionaries that map trackable_values to the corresponding feature value that can be fed to the model

__Exercise__

In [50]:
# Make a dictionary of the exercise rankings and corresponding trackable_name
exercise_dict = { "Rest":0, "Little":1, "Medium":2, "Sport":3, "Intense":4 }

__Stress__

In [51]:
# Make a dictionary of the stress rankings and corresponding trackable_names
stress_dict = {"0":["Relaxed", "Vacation", ], "1":["Anxiety", "Panic", "Stressed", "Depressed"]}

__Food__

In [52]:
# Make a list of all the foods that are in the 'Tag' trackable_type
tag_food = ['coffee','pizza','caffeine','dairy','salad','pasta','tea','chocolate','bread','donut','almonds'
            'gluten','vitamin c','apple','watermelon','porridge','alcohol','cereal','yogurt','olive oil',
            'red wine','banana','cheese','ice cream','no gluten','sugar','probiotic','soda','raspberries',
            'cauliflower','pie','milk','rice','bacon','decaf coffee','cookie','marijuana','eggs ','peanut butter',
            'Yoghurt','mango','almond milk','egg','smoothie','Egg Salad','Brown Rice','strawberries','probiotics',
            'avocado','lettuce','vegetables','popcorn','blueberries','sweet potato','ate dairy','turkey sandwich',
            'grapes','butter','green tea','beans','coconut milk','Gluten Free','french fries','ginger ale',
            'candy','asparagus','Meat','corn','beer','orange','Scrambled eggs','veggie pizza','spinach','honey',
            'drank alcohol','whole 30','green beans','antibiotics','gluten free bread','peanuts','Cake','cheerios',
            'tomato','oats','cashews','sushi','starch','ramen','olives','no dairy','thai food','pancakes',
            'diet coke','curry','grilled cheese','Black beans','pork','Lots of carbs','hot dog']

### Iterate through the rows of the dataframe to fill the data arrays

In [53]:
# Throw an error if "Weather" is one of the features, because this will require the next snippet of code to be modified
if "Weather" in FEATURES:
    raise Exception("Weather was chosen as an input feature. To accomodate this, change the code that adds info to the X_array.") 

In [ ]:
# iterate through the rows
for i, row in our_trackable_types_df.iterrows():
    user_id = our_trackable_types_df.at[i, "user_id"]
    date = our_trackable_types_df.at[i, "checkin_date"]
    track_type =  our_trackable_types_df.at[i, "trackable_type"]

    # Add info to X_array
    # Exercise
    if track_type  == "Exercise":
        exercise_name = our_trackable_types_df.at[i, "trackable_name"]
        exercise_index = FEATURES.index("Exercise")
        exericise_value = exercise_dict[exercise_name]
        X_array[user_id-1, date-1, exercise_index] = 0
        
    # Stress
    if track_type == "Stress":
        stress_name = our_trackable_types_df.at[i, "trackable_name"]
        stress_index = FEATURES.index("Stress")
        if stress_name in stress_dict["0"]:
            X_array[user_id-1, date-1, stress_index] = 0
        elif stress_name in stress_dict["1"]:
            X_array[user_id-1, date-1, stress_index] = 1
            
    # Food.............
    # TO DO
            
    # add info to Y_array, only if there is a value of 1 in one of the condition columns
    if track_type  == "Condition":
        for cond in LABELS:
            if our_trackable_types_df.at[i, cond] == 1:
                cond_array_index = LABELS.index(cond)
                Y_array[user_id-1, date-1, cond_array_index] = 1

_CHECK THAT THE EXERCISE IT WORKS FINE_...